# Chintan Chitroda
## Logistic Regression VS Random Forest Comparision
##### Note : Acurracy and F1 score can be increased with use of Boosting Algorithms.
##### The Notebook Visualize the data from Bank Marketing and predict if the customer will respond positively to the campaign or not, For the target variable is “Deposit”.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import os
warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_csv('/kaggle/input/bank-marketing-dataset/bank.csv')

In [ ]:
data

In [ ]:
data.columns

In [ ]:
data.deposit.value_counts()

In [ ]:
sns.heatmap(data.isnull())

#### NO null values

###### 1  -Describe the pdays column, make note of the mean, median and minimum values. Anything fishy in the values?

In [ ]:
data.describe()

In [ ]:
sns.violinplot(data['pdays'])

In [ ]:
print('median',data.pdays.median())
print('mean:',data.pdays.mean())
print('mode:',data.pdays.mode())

#### 2 -	Describe the pdays column again, this time limiting yourself to the relevant values of pdays. How different are the mean and the median values?

In [ ]:
data.pdays.value_counts()

In [ ]:
dataWOP = data[data.pdays != -1]

In [ ]:
sns.violinplot(dataWOP.pdays)

In [ ]:
print('median',dataWOP.pdays.median())
print('mean:',dataWOP.pdays.mean())
print('mode:',dataWOP.pdays.mode())

##### As there are no values for customers who were not approached ( -1). The mean median and mode have changed significantly.

###### 3 -	Plot a horizontal bar graph with the median values of balance for each education level value. Which group has the highest median?

In [ ]:
data.balance.describe()

In [ ]:
data.columns

In [ ]:
data.education.value_counts()

In [ ]:
sns.barplot(data.balance,data.education)

######  4 - Make a box plot for pdays. Do you see any outliers?

In [ ]:
sns.boxplot(data.pdays)
print('outliers with -1')

In [ ]:
sns.boxplot(dataWOP.pdays)
print('outliers without -1')

#### Yes we can seee there are outliers in pdays

### EDA

In [ ]:
plt.figure(figsize=(15,15))
sns.heatmap(data.corr(),square=True,annot=True,cmap= 'twilight_shifted')

In [ ]:
data.info()

In [ ]:
data.replace({'deposit': {"yes": 1,'no':0}},inplace=True)

In [ ]:
### identigyind categorical and numerical columns
cols = data.columns
num_cols= data._get_numeric_data().columns
cat_cols = list(set(cols) - set(num_cols))

### Visualizing Categorical Features with Deposit

In [ ]:
for i in data[cat_cols]:
    sns.barplot(data.deposit,data[i])
    plt.show()

###  Visualizing Numerical Features with Deposit

In [ ]:
for i in data[num_cols]:
    sns.barplot(data.deposit,data[i])
    plt.show()

In [ ]:
cols = data.columns
num_cols= data._get_numeric_data().columns
cat_cols = list(set(cols) - set(num_cols))

In [ ]:
data[cat_cols]

In [ ]:
data[num_cols]

## PreProcessing

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
data1 = data[cat_cols].apply(LabelEncoder().fit_transform)

In [ ]:
data1

In [ ]:
dataf = data1.join(data[num_cols])

In [ ]:
dataf

In [ ]:
dataf[dataf.pdays == -1]

In [ ]:
dataf.pdays.value_counts()

### More than 70% of pdays have -1.

In [ ]:
#dataf.drop('pdays',axis=1)

##### After analyzing the we know that Pdays dones play any important role in model so we keep it as it is.
##### The -1 or the missing values are kept as it is as we would not be using that in modelling part

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(dataf.corr(),square=True,annot=True,cmap= 'Spectral')

In [ ]:
dataf.corr()

## ML Modelling

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
import statsmodels.api as sm
from sklearn.metrics import accuracy_score

### All Single feature with Logistic Regression

In [ ]:
for i in dataf.columns:
    X = dataf[[i]]
    y = dataf['deposit']
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,random_state=101)
    lr = LogisticRegression()
    lr.fit(X_train,y_train)
    y_pred = lr.predict(X_test)
    print("F1 Score for", i , f1_score(y_pred,y_test))

## All Features

In [ ]:
X = dataf.drop('deposit',axis=1)
y = dataf['deposit']

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,random_state=101)

In [ ]:
lr = LogisticRegression(random_state=101)
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)

In [ ]:
print('Report:\n',classification_report(y_test, y_pred))
print("F1 Score:",f1_score(y_pred,y_test))
print('confusion Matrix:\n',confusion_matrix(y_pred,y_test))
print('cross validation:',cross_val_score(lr, X, y, cv=5))

In [ ]:
model = sm.OLS(y, X)
results = model.fit()
start = "\033[1m" ### for bold text
print(start)
print(results.summary())

## Using RFE

In [ ]:
### Using Rfe
from sklearn.feature_selection import RFE
rfe = RFE(lr, 15)
rfe.fit(X_train,y_train)

In [ ]:
print(X_train.columns[rfe.support_])
cols = X_train.columns[rfe.support_]
lr.fit(X_train[cols],y_train)
y_pred2 = lr.predict(X_test[cols])

In [ ]:
def logReggResult():
    print('Report:\n',classification_report(y_test, y_pred2))
    print("F1 Score:",f1_score(y_pred2,y_test))
    print('AUC score:',roc_auc_score(y_test,y_pred2))
    print('confusion Matrix:\n',confusion_matrix(y_pred2,y_test))
    print('kfold cross validation:\n',cross_val_score(lr, X, y, cv=5))
    print("Acurracy :::>",accuracy_score(y_pred2,y_test))

In [ ]:
logReggResult()

In [ ]:
## Feature Importance
from sklearn.feature_selection import SelectFromModel
smf = SelectFromModel(lr)
smf.fit(X_train,y_train)
features = smf.get_support()
feature_name = X_train.columns[features]
feature_name

In [ ]:
importances=feature_name
feature_importances=pd.Series(importances).sort_values(ascending=False)
sns.barplot(x=feature_importances[0:10], y=feature_importances.index[0:10])
plt.title('Feature Importance',size=20)
plt.ylabel("Features")
plt.show()

## Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X = dataf.drop('deposit',axis=1)
y = dataf['deposit']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,random_state=101)

In [ ]:
rfc = RandomForestClassifier(n_estimators=50,max_depth=5, random_state=101,max_leaf_nodes=50)
rfc.fit(X_train,y_train)

In [ ]:
y_pred1 = rfc.predict(X_test)

In [ ]:
from sklearn.tree import export_graphviz
estimator = rfc.estimators_[5]
# Export as dot file
export_graphviz(estimator, 
                out_file='tree1.dot', 
                feature_names = X_train.columns,
                class_names = 'deposit',
                rounded = True, proportion = False, 
                precision = 2, filled = True)

In [ ]:
os.system('dot -Tpng tree1.dot -o tree1.png')

In [ ]:
from IPython.display import Image
Image(filename = 'tree1.png')

In [ ]:
def RF():
    print('Report:\n',classification_report(y_test, y_pred1))
    print("F1 Score:",f1_score(y_pred1,y_test))
    print('confusion Matrix:\n',confusion_matrix(y_pred1,y_test))
    print('cross validation:',cross_val_score(rfc, X, y, cv=5))
    print('AUC score:',roc_auc_score(y_test,y_pred))
    print("Acurracy :::>",accuracy_score(y_pred,y_test))

In [ ]:
RF()

In [ ]:
importances=rfc.feature_importances_
feature_importances=pd.Series(importances, index=X_train.columns).sort_values(ascending=False)
sns.barplot(x=feature_importances[0:10], y=feature_importances.index[0:10])
plt.title('Feature Importance',size=20)
plt.ylabel("Features")
plt.show()

# Comparision 

### We choose the F1 Score and Confustion Matrix for the Final comparision of both Models because the Target Variable is imbalanced. So, Precision and Recall can be the main factor for evaluation model and we get the harmonic mean for the same in form of F1 score.

In [ ]:
print (start + "############### Random Forest Result: ###############\n")
RF()
print(start +'\n\n############# Logistic Regression Result: ############\n')
logReggResult()

### We see that the Random Forest has Perfomed good in comparision to Logistic Regression.

### Random Forest has Better F1 score,  AUC,  Confusion Matrix, kfold Cross val and accuracy in compare to Logistic Regression.

### The Main Features for the both Models/ Algorithm Differs as Random Forest and Logistic Regression Works on different Principal

### Please do Upvote if you like kernal.